# Retail Transaction Data Cleaning

## 1. Load Data

In [1]:
import pandas as pd

df_2009 = pd.read_excel("data/retail_data.xlsx", sheet_name="Year 2009-2010")
df_2010 = pd.read_excel("data/retail_data.xlsx", sheet_name="Year 2010-2011")

df = pd.concat([df_2009, df_2010], ignore_index=True)

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 2. Inspect Columns and Basic Info

In [2]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

## 3. Rename Columns

In [3]:
df.columns = [
    "invoice_id",
    "stock_id",
    "description",
    "quantity",
    "invoice_date",
    "price",
    "customer_id",
    "country"
]

## 4. Standardize IDs

In [4]:
df["invoice_id"] = df["invoice_id"].astype(str).str.strip()
df["stock_id"] = df["stock_id"].astype(str).str.strip()

## 5. Remove Descriptions Starting with '*'

In [5]:
df = df[~df["description"].str.startswith("*", na=False)]

## 6. Trim Description Field

In [6]:
df["description"] = df["description"].str.strip()

## 7. Explore Unique Invoice Prefixes

In [7]:
df["invoice_id"].str[0].value_counts()

invoice_id
5    939378
4    108487
C     19494
A         6
Name: count, dtype: int64

## 8. Classify Transactions

In [8]:
df["transaction_type"] = "sale"

# Credit notes
df.loc[df["invoice_id"].str.startswith("C", na=False), "transaction_type"] = "credit note"

# Inventory corrections
mask_inventory = (
    df["invoice_id"].str[0].str.isdigit() &
    (df["quantity"] < 0) &
    (df["price"] == 0) &
    df["customer_id"].isna()
)
df.loc[mask_inventory, "transaction_type"] = "inventory correction"

# Unknown items
df.loc[df["description"].str.fullmatch(r"\?+", na=False), "transaction_type"] = "unknown item"

# StockID mapping
stock_map = {
    "B": "adjust bad debt",
    "S": "sample",
    "POST": "postage",
    "M": "manual",
    "DOT": "dotcom postage",
    "D": "discount",
    "CRUK": "cruk commission",
    "C3": "carriage",
    "C2": "carriage",
    "BANK CHARGES": "bank charges",
    "AMAZONFEE": "amazon fee",
    "ADJUST2": "adjustment",
    "ADJUST": "adjustment",
    "TEST001": "test",
    "TEST002": "test",
    "gift_0001_90": "gift card voucher -90",
    "gift_0001_80": "gift card voucher -80",
    "gift_0001_70": "gift card voucher -70",
    "gift_0001_60": "gift card voucher -60",
    "gift_0001_50": "gift card voucher -50",
    "gift_0001_40": "gift card voucher -40",
    "gift_0001_30": "gift card voucher -30",
    "gift_0001_20": "gift card voucher -20",
    "gift_0001_10": "gift card voucher -10",
}
df["transaction_type"] = df["stock_id"].map(stock_map).fillna(df["transaction_type"])


## 9. Parse Invoice Date

In [9]:
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")

## 10. Fill Missing Descriptions

In [10]:
desc_map = (
    df.loc[df["description"].notna()]
    .drop_duplicates(subset=["stock_id"])
    .set_index("stock_id")["description"]
)

df["description"] = df["description"].fillna(df["stock_id"].map(desc_map))

## 11. Remove Duplicates

In [11]:
df = df.drop_duplicates()

In [ ]:
## 12. Final Checks

In [12]:
df["transaction_type"].value_counts()
df.isna().sum()

invoice_id               0
stock_id                 0
description            366
quantity                 0
invoice_date             0
price                    0
customer_id         235146
country                  0
transaction_type         0
dtype: int64

## 13. Export Cleaned Data

In [13]:
df.to_csv("data/cleaned_retail_data.csv", index=False)